In [1]:
import numpy as np
from scipy import optimize
import itertools

In [2]:
def initialize_weights(L_in, L_out):
    e = 0.12 # sigma
    t = np.random.random((L_out, L_in + 1)) * 2 * e - e
    return t


def recode_label(y,num_labels):
    '''theres a chance this needs to be the other way'''
    rows = len(y)
    out = np.zeros((rows, num_labels))
    for i in range(0, rows):
        row_answer = int(y[i])
        out[i, row_answer] = 1
        
    return out

def param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels ):
    '''
    theta1 shape: (30, 785) (hidden_size, input_size + 1)
    theta2 shape: (26, 31)  (num_labels, hidden_size + 1)
    '''
    theta1_elems = ( input_layer_size + 1 ) * hidden_layer_size
    theta1_size  = ( input_layer_size + 1, hidden_layer_size  )
    theta1 = nn_params[:theta1_elems].T.reshape( theta1_size ).T

    

    theta2_size  = ( hidden_layer_size + 1, num_labels )
    theta2 = nn_params[theta1_elems:].T.reshape( theta2_size ).T

    return (theta1, theta2)

In [3]:
# Some math

def sigmoid(z):
    return ( (1 / (1 + np.exp(-z))) )

def sigmoid_gradient(z):
    return (sigmoid(z) * (1 - sigmoid(z)))

print(sigmoid(0.0)) #should return 0.5
print(sigmoid_gradient(0.0)) # should return 0.25

0.5
0.25


In [4]:
def feed_forward(theta1, theta2, X, X_bias=None):
    '''
    a1 = (m, input_layer_size + 1), a2 = (m, hidden_layer_size + 1), a3= (m, num_labels)
    theta1 = (hidden_layer_size, input_layer_size + 1)
    theta2 = (num_labels, hidden_layer_size)
    '''
    # Input layer
    m, _ = np.shape(X)
    one_rows = np.ones((1, np.shape(X)[0] ))
    a1 = np.c_[np.ones((m,1)), X] # assigning a1 to X, and adding a bias (m, input_layer_size + 1)
    # Hidden layer
    z2 = a1.dot(theta1.T)
    a2 = sigmoid(z2)
    a2 = np.c_[np.ones((np.shape(a2)[0], 1)), a2] # bias for hidden layer
    # Output layer
    z3 = a2.dot(theta2.T)
    a3 = sigmoid(z3) #a3 = h(x)
    
    return (a1, a2, a3, z2, z3)

In [5]:
def compute_cost( nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, _lambda, yk = None, X_bias = None ):
    
    theta1, theta2 = param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels )
    a1,a2,a3,z2,z3 = feed_forward(theta1, theta2, X, X_bias)
    
    if yk is None:
        yk = recode_label(y, num_labels)
        assert shape(yk) == shape(a3), "Error, shape of recoded y is different from a3"
    
    # J(theta) function: cross-entropy
    term1 = (-y_k * np.log(a3))
    term2 = (1 - y_k) * np.log(1 - a3)
    cost = np.sum(term1 + term2)/m
    # Regularization sum
    reg_term = np.sum(theta1 ** 2) + np.sum(theta2[:,1:] ** 2)
    reg_term = (_lambda/2/m) * reg_term
    return(cost + reg_term)

In [10]:
def compute_gradient( nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, _lambda, yk = None, X_bias = None ):
   
    m, n = np.shape(X)
    theta1, theta2 = param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels )
    a1, a2, a3, z2, z3 = feed_forward(theta1, theta2, X)
    

    # back propagate
    if yk is None:
        yk = recode_label(y, num_labels )
        assert shape(yk) == shape(a3), "Error: shape of recoded y is different from a3"
        
    delta_3 = a3 - y_k #(m, num_labels), theta2=(labels, hidden_label size)
    delta_2 = (delta_3.dot(theta2))[:,1:] * sigmoid_gradient(z2) #ignore bias
    sum_2 = delta_3.T.dot(a2) # sum of a_i * delta_i+1
    sum_1 = delta_2.T.dot(a1)
    
    # putting the gradient equation together
    theta_2_grad = (sum_2[:,1:] / m) + ((theta2[:,1:] * _lambda) / m)
    theta_1_grad = (sum_1[:,1:] / m) + (theta1[:,1:] * _lambda / m)
    
    # Sizes
    #grad1: (28, 784)
    #grad2: (26, 28)
    grad_flat = np.r_[theta_1_grad.T.flatten(), theta_2_grad.T.flatten()]
    return grad_flat


In [11]:
# loading input data
data = np.genfromtxt('./data/5.csv', delimiter=',')
y = data[:,0].reshape(-1,1)
X = data[:, 1:] #(m, input_layer_size)
m = len(y)

# Network architecture 
input_layer_size = 784
hidden_layer_size = 30
num_labels = 26
lam = 1.0

# Params
theta1 = initialize_weights( 784, 30 ) # (input_size, hidden layer 1 size)
theta2 = initialize_weights( 30, 26 )  # (hidden layer 1 size, # labels)
unrolled = np.r_[theta1.T.flatten(), theta2.T.flatten()] # 1 dimension (24356,)

# matrix holding correct values
y_k = recode_label(y,num_labels)

X_bias = np.r_[ np.ones((1, np.shape(X)[0] )), X.T].T #(input_size + 1, m)
print(np.shape(X_bias))


(5, 785)


In [12]:
# Debugging the parameters before
print(f'rolled params: {np.shape(unrolled)}')
print(f'number of elements in theta1: {np.size(theta1)}')
print(f'number of elements in theta2: {np.size(theta2)}')
print(f'theta1 shape: {np.shape(theta1)}')
print(f'theta2 shape: {np.shape(theta2)}')

print('='*20)

t1,t2 = param_unroll(unrolled, input_layer_size, hidden_layer_size, num_labels)

# Debugging after:
print(f'number of elements in theta1: {np.size(t1)}')
print(f'number of elements in theta2: {np.size(t2)}')
print(f'theta1 shape: {np.shape(t1)}')
print(f'theta2 shape: {np.shape(t2)}')

# debugging values are the same
print(f'{theta1[0,0] == t1[0,0]},{theta1[12,25] == t1[12,25]},{theta1[11, 200] == t1[11,200]}')
print(f'{theta2[0,0] == t2[0,0]},{theta2[12,25] == t2[12,25]},{theta2[11, 11] == t2[11,11]}')

rolled params: (24356,)
number of elements in theta1: 23550
number of elements in theta2: 806
theta1 shape: (30, 785)
theta2 shape: (26, 31)
number of elements in theta1: 23550
number of elements in theta2: 806
theta1 shape: (30, 785)
theta2 shape: (26, 31)
True,True,True
True,True,True


In [13]:

feed_forward(theta1,theta2, X, X_bias)
print(compute_cost(unrolled,input_layer_size, hidden_layer_size, num_labels, X, y, lam, y_k, X_bias))

result = optimize.fmin_cg(compute_cost, fprime=compute_gradient, x0=unrolled,
    args=(input_layer_size, hidden_layer_size, num_labels, X, y, lam, y_k, X_bias),
    maxiter=50, disp=True, full_output=True )


-5.460790016638789


ValueError: operands could not be broadcast together with shapes (24356,) (24300,) 